#  FrozenLake
Today you are going to learn how to survive walking over the (virtual) frozen lake through discrete optimization.

<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>


In [1]:
import gym

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

[2017-01-04 12:54:13,538] Making new env: FrozenLake-v0


In [2]:
# display the game state
env.render()

SFFF
FHFH
FFFH
HFFG



### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [3]:
print "initial observation code:",env.reset()
print 'printing observation:'
env.render()
print "observations:",env.observation_space, 'n=',env.observation_space.n
print "actions:",env.action_space, 'n=',env.action_space.n

SyntaxError: invalid syntax (<ipython-input-3-b4c1c61130ff>, line 1)

In [4]:
print "taking action 2 (right)"
new_obs, reward, is_done, _ = env.step(2)
print "new observation code:",new_obs
print "reward:", reward
print "is game over?:",is_done
print "printing new state:"
env.render()

SyntaxError: invalid syntax (<ipython-input-4-b82e6d054a9e>, line 1)

In [43]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [44]:
env.step(action_to_i['up'])
env.render()

SFFF
FHFH
FFFH
HFFG
  (Up)


### Policy

* The environment has a 4x4 grid of states (16 total), they are indexed from 0 to 15
* From each states there are 4 actions (left,down,right,up), indexed from 0 to 3

We need to define agent's policy of picking actions given states. Since we have only 16 disttinct states and 4 actions, we can just store the action for each state in an array.

This basically means that any array of 16 integers from 0 to 3 makes a policy.

In [45]:
import numpy as np
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.randint(0, 4, 16)

In [46]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]

assert all([len(p) == 16 for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == 3, 'maximal action id should be 3'
action_probas = np.unique(policies,return_counts=True)[-1] /10**4. /16.
print ("Action frequencies over 10^4 samples:"),action_probas
assert np.allclose(action_probas,[0.25]*4,atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print ("Seems fine!")

Action frequencies over 10^4 samples:
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [47]:
def sample_reward(env,policy,t_max=27):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    """
    s = env.reset()
    total_reward = 0
    
    for i in range(t_max):
        s, rev, is_end, _ = env.step(policy[s])
        total_reward += rev
        if is_end: break
    return total_reward

In [48]:
print ("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int,float) for r in rewards]), 'sample_reward must return a single number'
assert all([0 <= r <= 1 for r in rewards]), 'total rewards should be between 0 and 1 for frozenlake'
print ("Looks good!")

generating 10^3 sessions...
Looks good!


In [49]:
def evaluate(policy,n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for i in range(n_times)]
    return np.mean(rewards)
        

In [50]:
def print_policy(policy):
    """a function that displays a policy in a human-readable way"""
    lake = "SFFFFHFHFFFHHFFG"
    
    # where to move from each tile
    arrows = ['<v>^'[a] for a in policy]
    
    #draw arrows above S and F only
    signs = [arrow if tile in "SF" else tile for arrow,tile in zip(arrows,lake)]
    
    for i in range(0,16,4):
        print (' '.join(signs[i:i+4]))

print ("random policy:")
print_policy(get_random_policy())

random policy:
> < < >
v H > H
> ^ > H
H > < G


### Random search

In [51]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(10000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print ("New best score:",score)
        print ("Best policy:",
        print_policy(best_policy))


  0%|          | 5/10000 [00:00<06:41, 24.89it/s]

New best score: 0.0
> v > >
< H ^ H
^ < v H
H < < G
Best policy: None
New best score: 0.01
> < < >
< H ^ H
v < < H
H > ^ G
Best policy: None
New best score: 0.05
v ^ < <
< H v H
^ v ^ H
H > > G
Best policy: None



  0%|          | 10/10000 [00:00<08:26, 19.71it/s]

New best score: 0.06
< > v <
< H ^ H
^ ^ v H
H < > G
Best policy: None
New best score: 0.07
^ ^ > <
> H v H
^ < v H
H > v G
Best policy: None



  0%|          | 26/10000 [00:01<07:21, 22.57it/s]

...

New best score: 0.13
> ^ v ^
> H > H
v v v H
H > v G
Best policy: None


  3%|▎         | 267/10000 [00:11<07:42, 21.02it/s]

New best score: 0.27
< ^ < v
< H < H
^ v < H
H ^ v G
Best policy: None


  6%|▌         | 554/10000 [00:24<07:13, 21.80it/s]


KeyboardInterrupt: 

### Genetic algorithm

In [52]:
def crossover(policy1,policy2,p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    #policy = [int(np.random.choice([a, b], size=1, p = [p, 1-p])) for a, b in zip(policy1, policy2)]
    policy = np.choose(np.random.binomial(1, p, len(policy1)), [policy1, policy2])
    #policy = [policy1[i] if np.random.randint(0, 2, 1) == 1 else policy2[i] for i in range(len(policy1))]
    return policy

In [53]:
np.random.binomial(1, 0.1, 5)

array([0, 0, 0, 0, 0])

In [54]:
#test crossover
policy1, policy2 = [1, 2, 3], [4, 4, 5]
for i in range(10):
    print(crossover(policy1, policy2))
print(policy2)

[4 4 3]
[4 2 5]
[1 4 5]
[4 2 3]
[4 4 5]
[1 4 3]
[1 2 3]
[1 4 3]
[1 4 3]
[1 2 3]
[4, 4, 5]


In [55]:
np.random.seed(1234)
policies = [crossover(get_random_policy(),get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == 16 for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == 3, 'maximal action id should be 3'
print ("Seems fine!")

Seems fine!


In [75]:

n_epochs = 100 #how many cycles to make
pool_size = 200 #how many policies to maintain
n_crossovers = 80 #how many crossovers to make on each step
n_mutations = 30 #how many mutations to make on each tick


In [76]:
print ("initializing...")
pool = [get_random_policy() for i in range(pool_size)]
#pool_scores = <evaluate every policy in the pool, return list of scores>
pool_scores = [float(evaluate(p)) for p in pool]

initializing...


In [77]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float,int) for score in pool_scores])


In [78]:
def get_random_index(n):
    return np.random.randint(0, n, 1)

In [79]:
def mutate(policy):
    return crossover(policy, get_random_policy(), p = 0.5)

In [ ]:
#main loop
for epoch in range(n_epochs):
    print ("Epoch %s:"%epoch)
    
    crossovered = list(crossover(
                             pool[np.random.randint(0, len(pool), 1)],
                             pool[np.random.randint(0, len(pool), 1)]
                            )
                             for i in range(n_crossovers)
                  )
    mutated = list(mutate(pool[get_random_index(len(pool))])
                             for i in range(n_crossovers)
                  )
    
    assert type(crossovered) == type(mutated) == list
    
    #evaluate new scores
    new_policies = crossovered + mutated
    
    
    #add new sessions to the pool
    policies = pool + new_policies
    pool_scores = list(map(evaluate, policies))
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [policies[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]
    
    #print the best policy so far (last in ascending score order)
    print ("best score:",max(pool_scores))
    print_policy(pool[np.argmax(pool_scores)])

Epoch 0:


/Users/alexajax/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
/Users/alexajax/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


best score: 0.22
< > < <
< H ^ H
^ v v H
H v > G
Epoch 1:
best score: 0.27
< ^ v ^
< H < H
^ v < H
H v > G
Epoch 2:
best score: 0.25
< ^ v ^
< H < H
^ v < H
H v > G
Epoch 3:
best score: 0.34
< ^ v ^
< H < H
^ v < H
H v > G
Epoch 4:
best score: 0.27
> ^ < >
< H > H
^ v v H
H > v G
Epoch 5:
best score: 0.24
< ^ > ^
< H v H
^ v < H
H > > G
Epoch 6:
best score: 0.31
< ^ < >
< H > H
^ v < H
H > > G
Epoch 7:
best score: 0.3
< v v ^
< H > H
^ v v H
H > > G
Epoch 8:
best score: 0.3
< ^ > ^
< H v H
^ v < H
H > > G
Epoch 9:
best score: 0.3
< ^ < ^
< H > H
^ v > H
H > > G
Epoch 10:
best score: 0.31
v ^ > ^
< H < H
^ v v H
H > v G
Epoch 11:
best score: 0.39
< ^ v v
< H < H
^ v < H
H > > G
Epoch 12:
best score: 0.35
< ^ > >
< H > H
^ v < H
H > v G
Epoch 13:
best score: 0.39
< < > >
< H v H
^ v < H
H > > G
Epoch 14:
best score: 0.39
< ^ v ^
< H < H
^ v v H
H > v G
Epoch 15:
best score: 0.4
< ^ v v
< H v H
^ v v H
H > v G
Epoch 16:
best score: 0.4
< ^ > ^
< H v H
^ v < H
H > > G
Epoch 17:
best score:

## moar

The parameters of the genetic algorithm aren't optimal, try to find something better. (size, crossovers and mutations)

Try alternative crossover and mutation strategies
* prioritize crossover over higher-scores
* only mutate several random actions from existing policy, not the whole.
* try to select a more diverse pool, not just best scorers

See which combination works best!